In [1]:
from autogen_agentchat.agents import ToolUseAssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat, StopMessageTermination
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."


from model import create_clients
client = create_clients(0)[2]

get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

assistant = ToolUseAssistantAgent(
    "Weather_Assistant",
    model_client=client,
    registered_tools=[get_weather_tool],
)


['llama-3.1-70b-versatile', 'llama3-groq-70b-8192-tool-use-preview', 'llama3-70b-8192', 'mixtral-8x7b-32768']
llama-3.1-70b-versatile
Successfully initialized client with model: llama-3.1-70b-versatile
llama3-groq-70b-8192-tool-use-preview
Successfully initialized client with model: llama3-groq-70b-8192-tool-use-preview
llama3-70b-8192
Successfully initialized client with model: llama3-70b-8192
mixtral-8x7b-32768
Successfully initialized client with model: mixtral-8x7b-32768


In [2]:
team = RoundRobinGroupChat([assistant])
result = await team.run("What's the weather in New York?", termination_condition=StopMessageTermination())

In [3]:
result

TeamRunResult(messages=[TextMessage(source='user', content="What's the weather in New York?"), StopMessage(source='Weather_Assistant', content='The weather in New York is 72 degrees and Sunny. TERMINATE')])

In [4]:
result.messages[1].content

'The weather in New York is 72 degrees and Sunny. TERMINATE'

In [5]:
# pip install langchain, langchain-community, wikipedia , autogen-ext

import wikipedia
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(wiki_client=wikipedia, top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

langchain_wikipedia_tool = LangChainToolAdapter(tool)

In [6]:
wikipedia_assistant = ToolUseAssistantAgent(
    "WikiPedia_Assistant",
    model_client=client,
    registered_tools=[langchain_wikipedia_tool],
)
team = RoundRobinGroupChat([wikipedia_assistant])
result = await team.run(
    "Who was the first president of the United States?", termination_condition=StopMessageTermination()
)

In [7]:
result.messages[1].content

'George Washington was the first president of the United States.'

In [8]:
from autogen_agentchat.agents import CodeExecutorAgent, CodingAssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat, StopMessageTermination
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_ext.code_executor.docker_executor import DockerCommandLineCodeExecutor

async with DockerCommandLineCodeExecutor(work_dir="coding") as code_executor:  # type: ignore[syntax]
    code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)
    coding_assistant_agent = CodingAssistantAgent(
        "coding_assistant", model_client=client,
    )
    group_chat = RoundRobinGroupChat([coding_assistant_agent, code_executor_agent])
    result = await group_chat.run(
        task="Create a plot of NVDIA and TSLA stock returns YTD from 2024-01-01 and save it to 'nvidia_tesla_2024_ytd.png'.",
        termination_condition=StopMessageTermination(),
    )
    print(result)

Error processing publish message
Traceback (most recent call last):
  File "/home/tjamil/Dev/GenAI/Autogen0.4/autogen_core/application/_single_threaded_agent_runtime.py", line 385, in _process_publish
    _all_responses = await asyncio.gather(*responses)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tjamil/Dev/GenAI/Autogen0.4/autogen_core/application/_single_threaded_agent_runtime.py", line 376, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tjamil/.local/share/virtualenvs/Dev-A_vYTw5b/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 49, in on_message
    return await super().on_message(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tjamil/Dev/GenAI/Autogen0.4/autogen_core/components/_routed_agent.py", line 468, in on_message
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tjamil/Dev/GenA

TeamRunResult(messages=[TextMessage(source='user', content="Create a plot of NVDIA and TSLA stock returns YTD from 2024-01-01 and save it to 'nvidia_tesla_2024_ytd.png'."), TextMessage(source='coding_assistant', content="I'd be happy to help you with that. To create a plot of NVDIA and TSLA stock returns YTD from 2024-01-01, I'll need to use Python with the necessary libraries.\n\nHere's the Python code to accomplish this task:\n```\n# filename: stock_returns_plot.py\nimport yfinance as yf\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Define the stock tickers and the start date\ntickers = ['NVIDIA', 'Tesla']\nstart_date = '2024-01-01'\n\n# Download the data\ndata = yf.download(tickers, start=start_date)['Adj Close']\n\n# Calculate the daily returns\nreturns = data.pct_change()\n\n# Plot the returns\nplt.figure(figsize=(12, 6))\nfor ticker in tickers:\n    plt.plot(returns[ticker], label=ticker)\nplt.title('NVIDIA and Tesla Stock Returns YTD 2024')\nplt.xlabel('Date')\nplt.

In [9]:
result.messages[1].content

"I'd be happy to help you with that. To create a plot of NVDIA and TSLA stock returns YTD from 2024-01-01, I'll need to use Python with the necessary libraries.\n\nHere's the Python code to accomplish this task:\n```\n# filename: stock_returns_plot.py\nimport yfinance as yf\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Define the stock tickers and the start date\ntickers = ['NVIDIA', 'Tesla']\nstart_date = '2024-01-01'\n\n# Download the data\ndata = yf.download(tickers, start=start_date)['Adj Close']\n\n# Calculate the daily returns\nreturns = data.pct_change()\n\n# Plot the returns\nplt.figure(figsize=(12, 6))\nfor ticker in tickers:\n    plt.plot(returns[ticker], label=ticker)\nplt.title('NVIDIA and Tesla Stock Returns YTD 2024')\nplt.xlabel('Date')\nplt.ylabel('Returns')\nplt.legend()\nplt.savefig('nvidia_tesla_2024_ytd.png')\nplt.show()\n```\nPlease save this code in a file named `stock_returns_plot.py` and execute it. The script will download the necessary data, calcu

In [10]:
from IPython.display import Image

Image(filename="coding/nvidia_tesla_2024_ytd.png")  # type: ignore

FileNotFoundError: [Errno 2] No such file or directory: 'coding/nvidia_tesla_2024_ytd.png'